In [1]:
#Importo las librerías a usar
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max.columns', 200)
pd.set_option('display.max.rows', 200)

# ETL datasets asociados a cambios climaticos

-- Nota recomiendo para el pipelines solo automatizar para ese primer csv la eliminación de Message y Start_Date dado que se repite la información el resto de los datos a mi parecer están bien correlacionados

In [3]:
#Traigo los archivos a evaluar
df_calidadDelAire = pd.read_csv(r"..\Datasets\Air_Quality.csv")
df_energia = pd.read_csv(r"..\Datasets\energy.csv")
df_contaminacion_sonora = pd.read_csv(r'..\Datasets\sonidos_NY.csv')

In [4]:
df_calidadDelAire.info() #Evaluo la informacion existente

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16122 entries, 0 to 16121
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unique ID       16122 non-null  int64  
 1   Indicator ID    16122 non-null  int64  
 2   Name            16122 non-null  object 
 3   Measure         16122 non-null  object 
 4   Measure Info    16122 non-null  object 
 5   Geo Type Name   16122 non-null  object 
 6   Geo Join ID     16122 non-null  int64  
 7   Geo Place Name  16122 non-null  object 
 8   Time Period     16122 non-null  object 
 9   Start_Date      16122 non-null  object 
 10  Data Value      16122 non-null  float64
 11  Message         0 non-null      float64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.5+ MB


In [5]:
#El total de entradas de la ultima columna están vacíos, el resto cuenta con los datos
df_calidadDelAire.isnull().sum()
#df_calidadDelAire.shape = (16122, 12) tenemos 16.122 filas y 12 columnas

Unique ID             0
Indicator ID          0
Name                  0
Measure               0
Measure Info          0
Geo Type Name         0
Geo Join ID           0
Geo Place Name        0
Time Period           0
Start_Date            0
Data Value            0
Message           16122
dtype: int64

In [6]:
df_calidadDelAire.head(5)

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.64,NaN
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,06/01/2014,33.22,NaN
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,06/01/2013,31.25,NaN
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,06/01/2014,31.15,NaN
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,12/01/2008,5.89,NaN


In [7]:
#Nota: Chicos como les comenté anteriormente el problema de las fechas entonces solventé asi:

# 1.- Genero una funcion que detecte expresiones regulares para extraer solo la fecha y anexarla a otra columna llamada Final_Date
# 2.- Genero la columna de estaciones, sin embargo me encuentro con el problema de la fecha de inicio
# 3.- Por eso notarán que genero mas funciones para lograr extraer directamente el año

import re #Traigo la librería para buscar expresiones regulares

# Función para extraer la estación del año y dejarla en una nueva columna 'Stations'
def extract_season(year_str):
    match = re.search(r'(Summer|Winter)', year_str)
    if match:
        return match.group(0)
    else:
        return 'Annual'  # Si no se encuentra la estación, asumimos que es un dato anual

# Aplicamos la función a la columna 'Time Period' y almacenamos los resultados en la nueva columna 'Stations'
df_calidadDelAire['Stations'] = df_calidadDelAire['Time Period'].apply(extract_season)



In [8]:
df_calidadDelAire.head(1)

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message,Stations
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.64,NaN,Summer


In [9]:
# Función para extraer el año
def extract_year(period):
    match = re.search(r'\d{2}$', period)  # Expresión regular para encontrar los últimos dos números
    if match:
        return f'20{match.group()}'
    else:
        return None

# Aplicar la función a la columna "Time Period" y crear una nueva columna "Final Date"
df_calidadDelAire['Final Date'] = df_calidadDelAire['Time Period'].apply(extract_year)

# Eliminar la columna "Year" si ya no se necesita
df_calidadDelAire.drop(columns=['Time Period'], inplace=True)

In [10]:
df_calidadDelAire.head(1)

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Start_Date,Data Value,Message,Stations,Final Date
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),06/01/2013,34.64,NaN,Summer,2013


In [11]:
#Elimino la columna que no tiene ningun dato representativo
df_calidadDelAire.drop(columns=['Message', 'Start_Date', 'Unique ID', 'Indicator ID', 'Geo Join ID'], inplace=True)

In [12]:
#Verifico que los datos estén quedando bien
df_calidadDelAire.head(5)

,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Data Value,Stations,Final Date
0,Ozone (O3),Mean,ppb,CD,Coney Island (CD13),34.64,Summer,2013
1,Ozone (O3),Mean,ppb,CD,Coney Island (CD13),33.22,Summer,2014
2,Ozone (O3),Mean,ppb,Borough,Bronx,31.25,Summer,2013
3,Ozone (O3),Mean,ppb,Borough,Bronx,31.15,Summer,2014
4,Sulfur Dioxide (SO2),Mean,ppb,CD,Morris Park and Bronxdale (CD11),5.89,Winter,2009


In [13]:
df_calidadDelAire = df_calidadDelAire[df_calidadDelAire['Geo Type Name'].isin(['Borough', 'Citywide'])]
#Nos quedamos solo con Borough y Citywide

In [14]:
df_calidadDelAire['Geo Type Name'].unique()

array(['Borough', 'Citywide'], dtype=object)

In [15]:
df_calidadDelAire = df_calidadDelAire.sort_values(by='Name', ascending=True)

In [16]:
df_calidadDelAire.to_csv(r'..\Datasets_procesados_contaminacion\Calidad del aire.csv', index=False) # Genero los datos a un csv

---

In [17]:
# Inicio con el segundo dataset sin embargo noto que contiene datos de muchos países y años por lo cual utilizo un filtro

df_energia = df_energia.loc[(df_energia['Country'] == 'United States') & (df_energia['Year'] >= 2000)]

In [18]:
df_energia.isnull().sum() #Notese que pese a que no salen valores nulos en la columna CO2_emission hay valores 0
#Son 41 datos con numero 0, no están asociado a uso de energías que generen emision de CO2
# df_filtrado.shape --> (240, 11)

Unnamed: 0                     0
Country                        0
Energy_type                    0
Year                           0
Energy_consumption             0
Energy_production              0
GDP                            0
Population                     0
Energy_intensity_per_capita    0
Energy_intensity_by_GDP        0
CO2_emission                   0
dtype: int64

In [19]:
df_energia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 29040 to 55379
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   120 non-null    int64  
 1   Country                      120 non-null    object 
 2   Energy_type                  120 non-null    object 
 3   Year                         120 non-null    int64  
 4   Energy_consumption           120 non-null    float64
 5   Energy_production            120 non-null    float64
 6   GDP                          120 non-null    float64
 7   Population                   120 non-null    float64
 8   Energy_intensity_per_capita  120 non-null    float64
 9   Energy_intensity_by_GDP      120 non-null    float64
 10  CO2_emission                 120 non-null    float64
dtypes: float64(7), int64(2), object(2)
memory usage: 11.2+ KB


In [20]:
df_energia.head(3)

,Unnamed: 0,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
29040,29040,United States,all_energy_types,2000,98.702224,71.270590,13754.31,282054.2,349.940629,7.176094,5700.252865
29041,29041,United States,coal,2000,22.579528,22.735478,13754.31,282054.2,349.940629,7.176094,2061.622698
29042,29042,United States,natural_gas,2000,23.823977,19.661529,13754.31,282054.2,349.940629,7.176094,1196.935249


In [21]:
#El Energy_intensity_by_GDP es una medida  que sirve para correlacionar el crecimiento del PIB con el gasto de energía de un país a menor consumo 
#de GDP mejor porque significa mejor uso de recursos energeticos


# df_filtrado['Energy_intensity_by_GDP'].min() #Se corresponde con el año 2019

In [22]:
df_energia.drop(columns='Unnamed: 0', inplace=True)

In [23]:
df_energia.to_csv('..\Datasets_procesados_contaminacion\Energia.csv', index=False) # Genero los datos a un csv

---

In [24]:
df_contaminacion_sonora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62022 entries, 0 to 62021
Data columns (total 78 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   split                                         62022 non-null  object 
 1   sensor_id                                     62022 non-null  int64  
 2   audio_filename                                62022 non-null  object 
 3   annotator_id                                  62022 non-null  int64  
 4   borough                                       62022 non-null  int64  
 5   block                                         62022 non-null  int64  
 6   latitude                                      62022 non-null  float64
 7   longitude                                     62022 non-null  float64
 8   year                                          62022 non-null  int64  
 9   week                                          62022 non-null 

In [25]:
df_contaminacion_sonora.shape

(62022, 78)

In [26]:
df_contaminacion_sonora.duplicated().sum()

0

In [27]:
df_contaminacion_sonora.head(5)

,split,sensor_id,audio_filename,annotator_id,borough,block,latitude,longitude,year,week,day,hour,1-1_small-sounding-engine_presence,1-2_medium-sounding-engine_presence,1-3_large-sounding-engine_presence,1-X_engine-of-uncertain-size_presence,2-1_rock-drill_presence,2-2_jackhammer_presence,2-3_hoe-ram_presence,2-4_pile-driver_presence,2-X_other-unknown-impact-machinery_presence,3-1_non-machinery-impact_presence,4-1_chainsaw_presence,4-2_small-medium-rotating-saw_presence,4-3_large-rotating-saw_presence,4-X_other-unknown-powered-saw_presence,5-1_car-horn_presence,5-2_car-alarm_presence,5-3_siren_presence,5-4_reverse-beeper_presence,5-X_other-unknown-alert-signal_presence,6-1_stationary-music_presence,6-2_mobile-music_presence,6-3_ice-cream-truck_presence,6-X_music-from-uncertain-source_presence,7-1_person-or-small-group-talking_presence,7-2_person-or-small-group-shouting_presence,7-3_large-crowd_presence,7-4_amplified-speech_presence,7-X_other-unknown-human-voice_presence,8-1_dog-barking-whining_presence,1-1_small-sounding-engine_proximity,1-2_medium-sounding-engine_proximity,1-3_large-sounding-engine_proximity,1-X_engine-of-uncertain-size_proximity,2-1_rock-drill_proximity,2-2_jackhammer_proximity,2-3_hoe-ram_proximity,2-4_pile-driver_proximity,2-X_other-unknown-impact-machinery_proximity,3-1_non-machinery-impact_proximity,4-1_chainsaw_proximity,4-2_small-medium-rotating-saw_proximity,4-3_large-rotating-saw_proximity,4-X_other-unknown-powered-saw_proximity,5-1_car-horn_proximity,5-2_car-alarm_proximity,5-3_siren_proximity,5-4_reverse-beeper_proximity,5-X_other-unknown-alert-signal_proximity,6-1_stationary-music_proximity,6-2_mobile-music_proximity,6-3_ice-cream-truck_proximity,6-X_music-from-uncertain-source_proximity,7-1_person-or-small-group-talking_proximity,7-2_person-or-small-group-shouting_proximity,7-3_large-crowd_proximity,7-4_amplified-speech_proximity,7-X_other-unknown-human-voice_proximity,8-1_dog-barking-whining_proximity,1_engine_presence,2_machinery-impact_presence,3_non-machinery-impact_presence,4_powered-saw_presence,5_alert-signal_presence,6_music_presence,7_human-voice_presence,8_dog_presence
0,test,0,00_026884.wav,-6,1,547,40.72951,-73.99388,2019,43,4,8,-1,-1,-1,-1,0,0,0,0,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,-1,-1
1,test,0,00_026884.wav,-4,1,547,40.72951,-73.99388,2019,43,4,8,0,0,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
2,test,0,00_026884.wav,-3,1,547,40.72951,-73.99388,2019,43,4,8,0,1,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
3,test,0,00_026884.wav,-1,1,547,40.72951,-73.99388,2019,43,4,8,-1,-1,-1,-1,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1
4,test,0,00_026884.wav,0,1,547,40.72951,-73.99388,2019,43,4,8,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0,1,0,0,0,0,0


In [28]:
df_contaminacion_sonora = df_contaminacion_sonora[['split','sensor_id','borough','year','5-1_car-horn_presence','5-2_car-alarm_presence',
                                                   '5-4_reverse-beeper_presence','1_engine_presence','2_machinery-impact_presence',
                                                   '3_non-machinery-impact_presence','4_powered-saw_presence','5_alert-signal_presence',
                                                   '6_music_presence','7_human-voice_presence','8_dog_presence']]


In [29]:
df_contaminacion_sonora.head()

,split,sensor_id,borough,year,5-1_car-horn_presence,5-2_car-alarm_presence,5-4_reverse-beeper_presence,1_engine_presence,2_machinery-impact_presence,3_non-machinery-impact_presence,4_powered-saw_presence,5_alert-signal_presence,6_music_presence,7_human-voice_presence,8_dog_presence
0,test,0,1,2019,-1,-1,-1,-1,1,0,-1,-1,-1,-1,-1
1,test,0,1,2019,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
2,test,0,1,2019,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
3,test,0,1,2019,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1
4,test,0,1,2019,0,0,0,1,0,1,0,0,0,0,0


In [30]:
df_contaminacion_sonora.isna().sum()   # Checkeamos Nulos por columna

split                              0
sensor_id                          0
borough                            0
year                               0
5-1_car-horn_presence              0
5-2_car-alarm_presence             0
5-4_reverse-beeper_presence        0
1_engine_presence                  0
2_machinery-impact_presence        0
3_non-machinery-impact_presence    0
4_powered-saw_presence             0
5_alert-signal_presence            0
6_music_presence                   0
7_human-voice_presence             0
8_dog_presence                     0
dtype: int64

In [31]:
df_contaminacion_sonora['borough'].unique()  
# Las localidades en NewYork estan mostradas como numeros
# Son 5 pero aqui solo tenemos 3
# (1=Manhattan, 3=Brooklyn, 4=Queens)

array([1, 3, 4], dtype=int64)

In [32]:
# Diccionario que mapea los números a los nombres correspondientes
borough_mapping = {
    1: 'Manhattan',
    3: 'Brooklyn',
    4: 'Queens'
}

# Reemplazar los números por los nombres usando el método 'replace'
df_contaminacion_sonora['borough'] = df_contaminacion_sonora['borough'].replace(borough_mapping)

In [33]:
df_contaminacion_sonora

,split,sensor_id,borough,year,5-1_car-horn_presence,5-2_car-alarm_presence,5-4_reverse-beeper_presence,1_engine_presence,2_machinery-impact_presence,3_non-machinery-impact_presence,4_powered-saw_presence,5_alert-signal_presence,6_music_presence,7_human-voice_presence,8_dog_presence
0,test,0,Manhattan,2019,-1,-1,-1,-1,1,0,-1,-1,-1,-1,-1
1,test,0,Manhattan,2019,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
2,test,0,Manhattan,2019,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
3,test,0,Manhattan,2019,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1
4,test,0,Manhattan,2019,0,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62017,validate,46,Manhattan,2018,1,0,0,0,0,0,0,1,0,0,0
62018,validate,46,Manhattan,2018,1,0,0,0,0,0,0,1,0,1,0
62019,validate,46,Manhattan,2018,0,0,0,0,0,0,0,0,0,1,0
62020,validate,46,Manhattan,2018,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# Supongamos que tienes un DataFrame llamado 'df' con las columnas originales
# y cargaste tus datos en el DataFrame 'df'

# Selecciona las columnas que deseas mantener
columns_to_keep = ['year', '5-1_car-horn_presence', '5-2_car-alarm_presence', '5-4_reverse-beeper_presence',
                   '1_engine_presence', '2_machinery-impact_presence', '3_non-machinery-impact_presence',
                   '4_powered-saw_presence', '5_alert-signal_presence', '6_music_presence', '7_human-voice_presence',
                   '8_dog_presence']

# Crea un nuevo DataFrame con las columnas seleccionadas
new_df = df_contaminacion_sonora[columns_to_keep]

# Renombra las columnas para quitar los números
new_columns = ['year', 'car-horn_presence', 'car-alarm_presence', 'reverse-beeper_presence',
               'engine_presence', 'machinery-impact_presence', 'non-machinery-impact_presence',
               'powered-saw_presence', 'alert-signal_presence', 'music_presence', 'human-voice_presence',
               'dog_presence']

new_df.columns = new_columns

# Ahora 'new_df' contiene solo las columnas seleccionadas y con nombres modificados


In [37]:
df_filtered = new_df

# Reemplazar -1 con 1 en todas las columnas excepto 'year'
df_filtered.iloc[:, 1:] = df_filtered.iloc[:, 1:].replace(-1, 1)

# Agrupar y sumar por año
summed_df = df_filtered.groupby('year').sum().reset_index()

C:\Users\Alan\AppData\Local\Temp\ipykernel_7472\2520156365.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.iloc[:, 1:] = df_filtered.iloc[:, 1:].replace(-1, 1)


In [39]:
# Convertir las columnas "tipo_sonido" en registros
summed_melted = pd.melt(summed_df, id_vars=['year'], var_name='tipo_sonido', value_name='conteo')

# Filtrar los valores diferentes de 0 en "conteo"
summed_melted = summed_melted[summed_melted['conteo'] != 0]

In [40]:
summed_melted

,year,tipo_sonido,conteo
0,2016,car-horn_presence,277
1,2017,car-horn_presence,3303
2,2018,car-horn_presence,2758
3,2019,car-horn_presence,3127
4,2016,car-alarm_presence,97
5,2017,car-alarm_presence,1272
6,2018,car-alarm_presence,934
7,2019,car-alarm_presence,2444
8,2016,reverse-beeper_presence,135
9,2017,reverse-beeper_presence,2125


In [ ]:
df_contaminacion_sonora = summed_melted
df_contaminacion_sonora.to_csv('..\Datasets_procesados_contaminacion\Sonido_presencia.csv', index = False)